Import statements

In [5]:
import torch
import os
import shutil

Load the drive folder containing all required files

In [6]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/            'Risiko!_Synthetic_Dataset_Creator.ipynb'
 backgrounds/          'Risiko! Test.ipynb'
 coco_risiko.yaml       Split_train_test_val.ipynb
 custom_yolo.yaml       synthetic_dataset/
 datasets/              synthetic_images/
 pre_trained_weights/   tanks_flags_detection.ipynb
 real_images/           test_example.txt


#### Clone the GitHub repository yolov5 and install requirements

In [7]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15978, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 15978 (delta 89), reused 118 (delta 75), pack-reused 15831
Receiving objects: 100% (15978/15978), 14.61 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (10958/10958), done.
/content/drive/MyDrive/Colab environments/Risiko! DL/yolov5


Model definition

In [8]:
# number of classes: 6 tanks + 6 flags = 12 total classes
NUM_CLASSES = 12

# model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, autoshape=False, force_reload=True, classes=NUM_CLASSES)

# set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# save the model to the desired device
model.to(device)


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-6-10 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Overriding model.yaml nc=80 with nc=12

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3 

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

#### Install Comet to get interesting feedbacks

In [ ]:
#!pip install comet_ml
#!export COMET_API_KEY=<Your API Key>

#### Move the $.yaml$ file inside the $yolov5$ folder

In [9]:
# current path is $yolov5 folder
current_path = os.getcwd()
# name of the file containing training specifications
filename = "custom_yolo.yaml"
# source path of the file
file_path = os.path.join(current_path, "..", filename)
# path to the folder where the file has to be moved
dest_path = os.path.join(current_path, filename)
# copy the file in $file_path to $dest_path
shutil.copy(file_path, dest_path)

'/content/drive/MyDrive/Colab environments/Risiko! DL/yolov5/custom_yolo.yaml'

#### Fine-tuning the model

In [10]:
!python train.py --img 640 --epochs 10 --cache ram --cache disk --data custom_yolo.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=custom_yolo.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/Colab': No such file or directory
YOLOv5 🚀 2023-6-10 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=